## find_data_splits_dir_unit_test_dev.ipynb

Created on: Thursday April 7th, 2022  
Created by: Jacob Alexander Rose  

In [1]:
%load_ext autoreload
%autoreload 2

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import pandas as pd
from pathlib import Path
from icecream import ic
from rich import print as pp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pytorch_lightning as pl
# from torchvision import transforms as T
# import torchmetrics
import argparse

import wandb
import matplotlib.pyplot as plt

from hydra.experimental import compose, initialize, initialize_config_dir
import hydra
from omegaconf import DictConfig, OmegaConf
from typing import *
default_reader = None

from imutils.ml.models.pl import classifier
from imutils.ml.utils.experiment_utils import configure_callbacks, configure_loggers, configure_trainer
# from imutils.big.datamodule import Herbarium2022DataModule, Herbarium2022Dataset
from imutils.ml.data.datamodule import *
from imutils.ml.utils.etl_utils import ETL
import imutils

# from dataclasses import dataclass, asdict, replace
# from typing import *
# from omegaconf import DictConfig, OmegaConf

Importing imutils


In [2]:
# ds = ExtantLeavesDataset(label_col='genus')

# dm = ExtantLeavesDataModule(label_col='genus')
dm = Herbarium2022DataModule(label_col='genus')

self.transform_cfg:


{'preprocess': {'train': {'resize': 512}, 'val': {'resize': 256}, 'test': {'resize': 256}}, 
'batch_transform': {'train': {'random_resize_crop': 224}, 'val': {'center_crop': 224}, 
'test': {'center_crop': 224}}, 'normalize': [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]}

self.normalize: [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]
self.normalize: [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]
self.normalize: [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]


ic| subset: 'train'
    num_samples: 671817
    num_batches: 5249
    self.num_classes: 15501
    self.batch_size: 128
ic| subset: 'val'
    num_samples: 167955
    num_batches: 1313
    self.num_classes: 15501
    self.batch_size: 128
ic| subset: 'test'
    num_samples: 210407
    num_batches: 1644
    self.num_classes: 15501
    self.batch_size: 128


In [3]:
dm.get_cfg()

Herbarium2022DataModuleConfig(catalog_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8', subset='train', label_col='genus', train_size=0.8, shuffle=True, seed=14, batch_size=128, num_workers=16, pin_memory=True, persistent_workers=False, transform_cfg={'preprocess': {'train': {'resize': 512}, 'val': {'resize': 256}, 'test': {'resize': 256}}, 'batch_transform': {'train': {'random_resize_crop': 224}, 'val': {'center_crop': 224}, 'test': {'center_crop': 224}}, 'normalize': [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]}, remove_transforms=False)

In [4]:
dm.cfg

Herbarium2022DataModuleConfig(catalog_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8', subset='train', label_col='genus', train_size=0.8, shuffle=True, seed=14, batch_size=128, num_workers=16, pin_memory=True, persistent_workers=False, transform_cfg={'preprocess': {'train': {'resize': 512}, 'val': {'resize': 256}, 'test': {'resize': 256}}, 'batch_transform': {'train': {'random_resize_crop': 224}, 'val': {'center_crop': 224}, 'test': {'center_crop': 224}}, 'normalize': [[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]]}, remove_transforms=False)

In [30]:
def find_data_splits_dir(source_dir: str,
						 train_size: float=0.7) -> Path:
    """
    Given a base path of `source_dir`, construct the correct data split dir path using chosen train_size.
    """

    if f"train_size-{train_size:.1f}" in str(source_dir):
        return source_dir
        # print(f"Returning in 1st block -> source_dir: {source_dir}")
    
    out_dir = Path(source_dir)
    
    train_split_subdir = out_dir.parts[-1] 
    if train_split_subdir.split("-")[0] == "train_size":
        if train_split_subdir.split("-")[0] != f"{train_size:.1f}":
            return Path(*out_dir.parts[:-1], f"train_size-{train_size:.1f}")
        return out_dir

    if "splits" not in out_dir.parts[-2:]:
        out_dir = out_dir / "splits"

    out_dir = out_dir / f"train_size-{train_size:.1f}"
    return out_dir


# print(f"Returning in last block -> out_dir: {out_dir}")

In [31]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8'
train_size=0.7

In [32]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.7


In [33]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8'
train_size=0.8

In [34]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8


In [35]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits'
train_size=0.7

In [36]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.7


In [37]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits'
train_size=0.8

In [38]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8


In [39]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs'
train_size=0.7

In [40]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.7


In [41]:
source_dir='/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs'
train_size=0.8

In [42]:
result = find_data_splits_dir(source_dir=source_dir,
                     train_size=train_size)

print(result)

/media/data_cifs/projects/prj_fossils/data/raw_data/herbarium-2022-fgvc9_resize-512/catalogs/splits/train_size-0.8
